# Limpieza base de datos covid-19 #

En este archivo se tomarán los productos serviciales de los archivos .csv de la base de datos del Repositorio Covid-19. Primero se hará la extracción de datos de aquellos archivos considerados necesarios o enriquecedores para la investogación en curso, finalmente se agregarán todos estos datos en un dataset unidos por su respectiva comuna.

In [1]:
import pandas as pd
import os
import zipfile

# Extracción de archivos #

Primero, extraeremos solo los productos del database que nos sirven para nuestra investigación en curso.

In [2]:
zip_path = "datos-covid-19.zip"
extract_path = "datos_covid_extraccion"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    all_files = zip_ref.namelist()
    productos = [
        "producto1", "producto38", "producto41", "producto65", 
        "producto66", "producto80", "producto84"
    ]
    
    archivos_a_extraer = []
    
    for producto in productos:
        patrones = [
            f"{producto}/",
            f"output/{producto}/", 
            f"datos-covid-19/{producto}/",
            f"datos-covid-19/output/{producto}/"
        ]
        
        for archivo in all_files:
            if any(patron in archivo for patron in patrones) and archivo.endswith('.csv'):
                archivos_a_extraer.append(archivo)        
    if not os.path.exists(extract_path):
        os.makedirs(extract_path)
    for archivo in archivos_a_extraer:
        try:
            zip_ref.extract(archivo, extract_path)
        except Exception as e:
            print('upis')

# Limpieza de archivos #

En esta sección iremos limpiando los datos para dejar los de la Región Metropolitana, por cada producto ya leído se armará un DataFrame con lo escencial para luego concatenar todo por comunas.

In [3]:
# Función para manejar los nombres de las comunas

def limpiar_comuna(nombre):
    nombre = nombre.strip().title()
    nombre = nombre.replace('Nunoa', 'Ñuñoa')
    nombre = nombre.replace('Penalolen', 'Peñalolén')
    nombre = nombre.replace('Peñalolen', 'Peñalolén')
    nombre = nombre.replace('Conchali', 'Conchalí')
    nombre = nombre.replace('Estacion Central', 'Estación Central')
    nombre = nombre.replace('San Joaquin', 'San Joaquín')
    nombre = nombre.replace('San Jose De Maipo', 'San José de Maipo')
    nombre = nombre.replace('San Ramon', 'San Ramón')
    nombre = nombre.replace('Maipu', 'Maipú')
    nombre = nombre.replace('Alhue', 'Alhué')
    nombre = nombre.replace('Curacavi', 'Curacaví')
    if 'Desconocido' in nombre or 'Total' in nombre:
        return None
    return nombre

In [4]:
# Producto 1: Se filtrará por los datos de la Región Metropolitana, dejando la cantidad de casos confirmados por año, por cada región.

csv_path = "datos_covid_extraccion/datos-covid-19/producto1/Covid-19_std.csv"
df = pd.read_csv(csv_path)
df_rm = df[df["Region"] == "Metropolitana"].copy()
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], dayfirst=True)
df_rm["Año"] = df_rm["Fecha"].dt.year
df_agrupado = df_rm.groupby(["Comuna", "Año"])["Casos confirmados"].sum().reset_index()
df_agrupado["Comuna"] = df_agrupado["Comuna"].apply(limpiar_comuna)
df_agrupado.to_csv('casos_cofirmados_comuna.csv', index=False)
df_agrupado.head(20)


,Comuna,Año,Casos confirmados
0,Alhué,2020,16778.0
1,Alhué,2021,70930.0
2,Alhué,2022,191187.0
3,Alhué,2023,5597.0
4,Buin,2020,160555.0
5,Buin,2021,898107.0
6,Buin,2022,2308531.0
7,Buin,2023,58051.0
8,Calera De Tango,2020,35802.0
9,Calera De Tango,2021,203809.0


In [5]:
# Producto 38: Fallecidos por comuna

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto38/CasosFallecidosPorComuna.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_long = df_rm.melt(
    id_vars=["Region", "Codigo region", "Comuna", "Codigo comuna", "Poblacion"],
    var_name="Fecha",
    value_name="Casos fallecidos"
)

df_long["Fecha"] = pd.to_datetime(df_long["Fecha"], errors="coerce")

df_long["Año"] = df_long["Fecha"].dt.year

df_long['Comuna'] = df_long["Comuna"].apply(limpiar_comuna)

df_final = (df_long.groupby(["Comuna", "Año"], as_index=False)["Casos fallecidos"].sum().rename(columns={"Casos fallecidos": "Total fallecidos"}))

df_final = df_final[df_final["Comuna"].notna() & ~df_final["Comuna"].str.contains("Total|Desconocido", case=False)]

df_final.to_csv('muertes_por_anio_comunal.csv', index=False)
df_final.head()

,Comuna,Año,Total fallecidos
0,Alhué,2020.0,83.0
1,Alhué,2021.0,541.0
2,Alhué,2022.0,1088.0
3,Alhué,2023.0,70.0
4,Buin,2020.0,1762.0


In [6]:
# Producto 41: transacciones BIP por comuna durante la pandemia

df = pd.read_csv('datos_covid_extraccion/datos-covid-19/producto41/BIPComuna_std.csv')

df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')

df['Año'] = df['Fecha'].dt.year

df['Comuna'] = df['Comuna'].apply(limpiar_comuna)

df_limpio = df.dropna(subset={'Comuna', 'Codigo comuna'})

df_final = (df_limpio.groupby(['Comuna', "Año"], as_index=False)['Transacciones'].sum())

df_final.to_csv('transacciones_bip_por_comuna.csv', index=False)
df_final.head(20)

,Comuna,Año,Transacciones
0,Cerrillos,2020,1616394.0
1,Cerro Navia,2020,1186714.0
2,Conchalí,2020,1641957.0
3,El Bosque,2020,926224.0
4,Estación Central,2020,5460067.0
5,Huechuraba,2020,776352.0
6,Independencia,2020,1503482.0
7,La Cisterna,2020,4298804.0
8,La Florida,2020,5380051.0
9,La Granja,2020,686554.0


In [7]:
# Producto 65: Positividas test covid por comuna

df = pd.read_csv('datos_covid_extraccion/datos-covid-19/producto65/PositividadPorComuna.csv')

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_long = df_rm.melt(
    id_vars=["Region", "Codigo region", "Comuna", "Codigo comuna", "Poblacion"],
    var_name="Fecha",
    value_name="Positividad"
)

df_long['Comuna'] = df_long['Comuna'].apply(limpiar_comuna)
df_long["Positividad"] = pd.to_numeric(df_long["Positividad"], errors="coerce")
df_long["Fecha"] = pd.to_datetime(df_long["Fecha"], errors="coerce")
df_long["Año"] = df_long["Fecha"].dt.year

df_long = df_long.drop(columns={"Codigo region", "Codigo comuna", "Fecha"})

df_final = (df_long.groupby(['Comuna', "Año"], as_index=False)['Positividad'].sum())

df_final.to_csv('positividad_test_covid19.csv', index=False)
df_final.head()

,Comuna,Año,Positividad
0,Alhué,2020,99.0
1,Alhué,2021,271.4
2,Alhué,2022,610.0
3,Buin,2020,90.1
4,Buin,2021,385.7


In [8]:
# Producto 66: cobertura de casos sospechosos con test

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto66/CoberturaPorComuna_std.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm['Comuna'] = df['Comuna'].apply(limpiar_comuna)
df_rm["Cobertura testeo"] = pd.to_numeric(df_rm["Cobertura testeo"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_promedio = (
    df_rm.groupby(["Comuna", "Año"], as_index=False)["Cobertura testeo"]
    .mean()
    .rename(columns={"Cobertura testeo": "Cobertura promedio"})
)

df_promedio.to_csv('cobertura_casos_sospechosos_con_test_prom.csv', index=False)
df_promedio.head()


,Comuna,Año,Cobertura promedio
0,Alhué,2020,89.584211
1,Alhué,2021,67.358824
2,Alhué,2022,88.466667
3,Buin,2020,90.894737
4,Buin,2021,88.646154


In [9]:
# Archivo 80: Avance primera dosis vacuna covid-19

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto80/vacunacion_comuna_1eraDosis_std.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm['Comuna'] = df_rm['Comuna'].apply(limpiar_comuna)

df_rm["Primera Dosis"] = pd.to_numeric(df_rm["Primera Dosis"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_primera_dosis = (df_rm.groupby(["Comuna", "Año"], as_index=False)["Primera Dosis"].sum())

df_primera_dosis.rename(columns={"Primera Dosis": "Vacunados 1ra dosis"}, inplace=True)

df_primera_dosis

,Comuna,Año,Vacunados 1ra dosis
0,Alhué,2020,0.0
1,Alhué,2021,7894.0
2,Alhué,2022,551.0
3,Alhué,2023,5.0
4,Buin,2020,0.0
...,...,...,...
203,Vitacura,2023,62.0
204,Ñuñoa,2020,75.0
205,Ñuñoa,2021,250953.0
206,Ñuñoa,2022,8280.0


In [10]:
# Producto 80: vacunados segunda dosis

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto80/vacunacion_comuna_2daDosis_std.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm['Comuna'] = df_rm['Comuna'].apply(limpiar_comuna)
df_rm["Segunda Dosis"] = pd.to_numeric(df_rm["Segunda Dosis"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_segunda_dosis = (df_rm.groupby(["Comuna", "Año"], as_index=False)["Segunda Dosis"].sum())

df_segunda_dosis.rename(columns={"Segunda Dosis": "Vacunados 2da dosis"}, inplace=True)

df_segunda_dosis

,Comuna,Año,Vacunados 2da dosis
0,Alhué,2020,0.0
1,Alhué,2021,7538.0
2,Alhué,2022,666.0
3,Alhué,2023,1.0
4,Buin,2020,0.0
...,...,...,...
203,Vitacura,2023,40.0
204,Ñuñoa,2020,0.0
205,Ñuñoa,2021,241571.0
206,Ñuñoa,2022,12539.0


In [11]:
# Producto 80: cuarta dosis cant vacunados

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto80/vacunacion_comuna_4taDosis_std.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm['Comuna'] = df_rm['Comuna'].apply(limpiar_comuna)
df_rm["Cuarta Dosis"] = pd.to_numeric(df_rm["Cuarta Dosis"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_cuarta_dosis = (df_rm.groupby(["Comuna", "Año"], as_index=False)["Cuarta Dosis"].sum())

df_cuarta_dosis.rename(columns={"Cuarta Dosis": "Vacunados 4ta dosis"}, inplace=True)

df_cuarta_dosis

,Comuna,Año,Vacunados 4ta dosis
0,Alhué,2020,0.0
1,Alhué,2021,0.0
2,Alhué,2022,6052.0
3,Alhué,2023,30.0
4,Buin,2020,0.0
...,...,...,...
203,Vitacura,2023,402.0
204,Ñuñoa,2020,0.0
205,Ñuñoa,2021,0.0
206,Ñuñoa,2022,184779.0


In [12]:
# Producto 80: dosis refuerzo cant vacunados

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto80/vacunacion_comuna_Refuerzo_std.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm['Comuna'] = df_rm['Comuna'].apply(limpiar_comuna)
df_rm["Dosis Refuerzo"] = pd.to_numeric(df_rm["Dosis Refuerzo"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_dosis_refuerzo = (df_rm.groupby(["Comuna", "Año"], as_index=False)["Dosis Refuerzo"].sum())

df_dosis_refuerzo.rename(columns={"Dosis Refuerzo": "Vacunados dosis de refuerzo"}, inplace=True)

df_dosis_refuerzo

,Comuna,Año,Vacunados dosis de refuerzo
0,Alhué,2020,0.0
1,Alhué,2021,5430.0
2,Alhué,2022,2516.0
3,Alhué,2023,7.0
4,Buin,2020,0.0
...,...,...,...
203,Vitacura,2023,158.0
204,Ñuñoa,2020,0.0
205,Ñuñoa,2021,173467.0
206,Ñuñoa,2022,64093.0


In [13]:
# Producto 80: unica dosis cant vacunados

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto80/vacunacion_comuna_UnicaDosis_std.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm['Comuna'] = df_rm['Comuna'].apply(limpiar_comuna)
df_rm["Unica Dosis"] = pd.to_numeric(df_rm["Unica Dosis"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_unica_dosis = (df_rm.groupby(["Comuna", "Año"], as_index=False)["Unica Dosis"].sum())

df_unica_dosis.rename(columns={"Unica Dosis": "Vacunados unica dosis"}, inplace=True)

df_unica_dosis

,Comuna,Año,Vacunados unica dosis
0,Alhué,2020,0.0
1,Alhué,2021,602.0
2,Alhué,2022,0.0
3,Alhué,2023,0.0
4,Buin,2020,0.0
...,...,...,...
203,Vitacura,2023,0.0
204,Ñuñoa,2020,0.0
205,Ñuñoa,2021,8339.0
206,Ñuñoa,2022,130.0


In [14]:
# Producto 80: vacunación total por comuna

df_vacunas = (
    df_primera_dosis
    .merge(df_segunda_dosis, on=["Comuna", "Año"], how="outer")
    .merge(df_dosis_refuerzo, on=["Comuna", "Año"], how="outer")
    .merge(df_cuarta_dosis, on=["Comuna", "Año"], how="outer")
    .merge(df_unica_dosis, on=["Comuna", "Año"], how="outer")
)

df_vacunas = df_vacunas.fillna(0)

df_vacunas.to_csv('cant_vacunados_distintas_dosis.csv', index=False)
df_vacunas.head(20)


,Comuna,Año,Vacunados 1ra dosis,Vacunados 2da dosis,Vacunados dosis de refuerzo,Vacunados 4ta dosis,Vacunados unica dosis
0,Alhué,2020,0.0,0.0,0.0,0.0,0.0
1,Alhué,2021,7894.0,7538.0,5430.0,0.0,602.0
2,Alhué,2022,551.0,666.0,2516.0,6052.0,0.0
3,Alhué,2023,5.0,1.0,7.0,30.0,0.0
4,Buin,2020,0.0,0.0,0.0,0.0,0.0
5,Buin,2021,98241.0,93820.0,63672.0,0.0,2613.0
6,Buin,2022,3959.0,6178.0,27184.0,67680.0,0.0
7,Buin,2023,37.0,35.0,88.0,272.0,0.0
8,Calera De Tango,2020,0.0,0.0,0.0,0.0,0.0
9,Calera De Tango,2021,28409.0,27165.0,20076.0,0.0,1746.0


In [15]:
# Producto 84 parte 1: fallecidos 2020-2021

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto84/fallecidos_comuna_edad_totales_std_A.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm['Comuna'] = df_rm['Comuna'].apply(limpiar_comuna)
df_rm["Total"] = pd.to_numeric(df_rm["Total"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_muertes_a = (df_rm.groupby(["Comuna", "Año"], as_index=False)["Total"].sum().rename(columns={"Total": "Fallecidos"}))

df_muertes_a.head()


,Comuna,Año,Fallecidos
0,Alhué,2020,6.0
1,Alhué,2021,8.0
2,Alhué,2022,4.0
3,Buin,2020,158.0
4,Buin,2021,132.0


In [16]:
# Producto 84 parte 2: total fallecidos 2022

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto84/fallecidos_comuna_edad_totales_std_B.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm['Comuna'] = df_rm['Comuna'].apply(limpiar_comuna)
df_rm["Total"] = pd.to_numeric(df_rm["Total"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_muertes_b = (df_rm.groupby(["Comuna", "Año"], as_index=False)["Total"].sum().rename(columns={"Total": "Fallecidos"}))

df_muertes_b.head()


,Comuna,Año,Fallecidos
0,Alhué,2022,4.0
1,Buin,2022,19.0
2,Calera De Tango,2022,6.0
3,Cerrillos,2022,26.0
4,Cerro Navia,2022,41.0


In [17]:
# Producto 84 final: Muertes 2020-2022

df_muertes = pd.concat([df_muertes_a, df_muertes_b], ignore_index=True)

df_muertes = df_muertes.groupby(["Comuna", "Año"], as_index=False)["Fallecidos"].sum()

df_muertes.to_csv('cant_muertes_20-21-22.csv', index=False)
df_muertes.head()


,Comuna,Año,Fallecidos
0,Alhué,2020,6.0
1,Alhué,2021,8.0
2,Alhué,2022,8.0
3,Buin,2020,158.0
4,Buin,2021,132.0


# Dataset con los datos filtrados #

En esta parte se juntarán todos los datos ya procesados en un solo archivo.

In [18]:
muertes_total = pd.read_csv('cant_muertes_20-21-22.csv')
cant_vacunados = pd.read_csv('cant_vacunados_distintas_dosis.csv')
casos_confirmados = pd.read_csv('casos_cofirmados_comuna.csv')
cobertura_sospechosos = pd.read_csv('cobertura_casos_sospechosos_con_test_prom.csv')
muertes = pd.read_csv('muertes_por_anio_comunal.csv')
positividad_test = pd.read_csv('positividad_test_covid19.csv')
transacciones_bip = pd.read_csv('transacciones_bip_por_comuna.csv')

# Renombramos algunas columnas para una mejor comprensión al leer el archivo .csv
muertes_total = muertes_total.rename(columns={'Fallecidos': 'Fallecidos por covid-19'})
positividad_test = positividad_test.rename(columns={'Positividad': 'Total de casos positivos según el test PCR'})
transacciones_bip = transacciones_bip.rename(columns={'Transacciones': 'Cant de transacciones BIP en 2020'})
cobertura_sospechosos = cobertura_sospechosos.rename(columns={'Cobertura promedio': 'Porcentaje promedio de cobertura de casos sospechosos'})

df_merge = casos_confirmados.merge(
    cobertura_sospechosos, on=['Comuna', 'Año'], how='outer').merge(
        positividad_test, on=['Comuna', 'Año'], how='outer').merge(
            cant_vacunados, on=['Comuna', 'Año'], how='outer').merge(
                muertes_total, on=['Comuna', 'Año'], how='outer').merge(
                    muertes, on=['Comuna', 'Año'], how='outer').merge(
                        transacciones_bip, on=['Comuna', 'Año'], how='outer')

df_merge = df_merge.dropna(subset=["Comuna"])

df_merge.to_csv('covid-19_datos_por_comuna.csv', index=False)
df_merge.head(20)

,Comuna,Año,Casos confirmados,Porcentaje promedio de cobertura de casos sospechosos,Total de casos positivos según el test PCR,Vacunados 1ra dosis,Vacunados 2da dosis,Vacunados dosis de refuerzo,Vacunados 4ta dosis,Vacunados unica dosis,Fallecidos por covid-19,Total fallecidos,Cant de transacciones BIP en 2020
0,Alhué,2020,16778.0,89.584211,99.0,0.0,0.0,0.0,0.0,0.0,6.0,83.0,NaN
1,Alhué,2021,70930.0,67.358824,271.4,7894.0,7538.0,5430.0,0.0,602.0,8.0,541.0,NaN
2,Alhué,2022,191187.0,88.466667,610.0,551.0,666.0,2516.0,6052.0,0.0,8.0,1088.0,NaN
3,Alhué,2023,5597.0,NaN,NaN,5.0,1.0,7.0,30.0,0.0,NaN,70.0,NaN
4,Buin,2020,160555.0,90.894737,90.1,0.0,0.0,0.0,0.0,0.0,158.0,1762.0,NaN
5,Buin,2021,898107.0,88.646154,385.7,98241.0,93820.0,63672.0,0.0,2613.0,132.0,8405.0,NaN
6,Buin,2022,2308531.0,93.041026,612.3,3959.0,6178.0,27184.0,67680.0,0.0,51.0,14491.0,NaN
7,Buin,2023,58051.0,NaN,NaN,37.0,35.0,88.0,272.0,0.0,NaN,896.0,NaN
8,Calera De Tango,2020,35802.0,90.952632,57.8,0.0,0.0,0.0,0.0,0.0,20.0,204.0,NaN
9,Calera De Tango,2021,203809.0,91.921154,365.8,28409.0,27165.0,20076.0,0.0,1746.0,38.0,1573.0,NaN
